In [1]:
from qiskit_utils import *

## **Quantum Palindrome Checking**

In tis problem our aim is to devise a quantum algorithm which can recognise whether a given string sequence is palindromic or not. However here I will try to elaborate a technique using which we can not only check classical strings but also more genralised string of quantum alphabets. 

For an quantum algorithm that solves this problem for calssical binary strings check out the \href{https://github.com/qosf/monthly-challenges}{qosf-monthly-challenges} repo.

### **Quantum Alphabets**

In the usual classical sense an alphabet could be any finite set distinguishable symbols, for eg. $\mathcal{A}:$ {`a`, `b`, `c`} or $\mathcal{A}:$ {`1`, `2`, `3`}, etc. To generalise this idea to include quantum states we can say than an *quantum alphabet* is a set of distinct of quantum states, for eg. $\mathcal{A}:$ {$\ket{\phi_1}$,$\ket{\phi_2}$,$\ket{\phi_3}$}. However note that that here 'distinct' doesn't necessarily mean orthogonal, so even while the quantum states are distinct in the sense that $ \ket{\phi_2} \neq \ket{\phi_1}$, they might not be classicaly distiguishable.

For eg. $\mathcal{A}:$ {$ \ket{\phi_1} = \: \ket{0} + e^{-i \frac{\pi}{2}} \ket{1}$, $\ket{\phi_2} = \:\ket{0} + e^{-i \pi}\ket{1}$, $\ket{\phi_3} = \: \ket{0} + e^{-i \frac{3\pi}{2}}\ket{1}$}

### **Palindrome Checking**

Ususally the strings we need to check would be of the strings of the form `1324321` or `abdcdab` ,etc. The usual way of handling them is to iteratively compare the subsquent bits of the original string against the reversed string. Since classical strings whatever be number or alphabets can be always decompsed into binary representations, it suffices to run the palindrome checking algorithm on the binary representation of the string. For eg. given a binary string `101100` we can compare it bitwise with the string `001101` to check if it is palindromic.

However given a string of quantum alphabet , say `str :`$\mathtt{\ket{\phi_2}\ket{\phi_1}\ket{\phi_1}\ket{\phi_2}}$ a qubit-wise comparison might not be very effecient or even possible given the non-orthogoal nature of the quantum alphabets. Thus we will try to use a different approach more sutied for quantum states and also highlight whether such an algorithm could have a quantum advantage.

#### **Algorithm**
Before moving onto the quantum algorithm there are some assumptions we need to make,
1. The length of the string needs to be a dyadic i.e a power of 2. Though this might not very          convenient we can always pad a regular string to this form. 
2. 